<h3>Demonstrate the use of MotifsFinder</h3>

In [1]:
import numpy as np
import pandas as pd
from Bio import Align
from Motifstools.FindMotifs import FindMotifs

Input examples. The files with the sequences must be in the Input directory in the format gene_name.fasta.
The genes in sample_list_at/vv are the one that sould be homologues from literature.
The task is one of the Phd thesis.

In [2]:
sample_input_list = ["meme_ara","meme_ara_1","meme_ara_2","meme_ara_3","meme_ara_4","meme_ara_5","meme_ara_6","meme_ara_7","meme_ara","meme_ara_1","meme_ara_2","meme_ara_3","meme_ara_4","meme_ara_5","meme_ara_6","meme_ara_7"]
sample_list_at = ["AT2G46680","AT4G26080","AT3G19290","AT1G08810","AT2G47160"]
sample_list_vv = ["VIT_15s0048g02870","VIT_11s0016g03180","VIT_18s0001g10450","VIT_08s0056g00800","VIT_17s0000g08530"]
sample_task_at = "AT4G28110"
sample_task_vv = ["VIT_12s0134g00570","VIT_19s0014g03820","VIT_00s0203g00070"]

Used for alignment

In [3]:
at_cdna = pd.read_csv("Input/arabidopsis_cDNA.csv",sep="\t")
vv_cdna = pd.read_csv("Input/vitis_cDNA.csv",sep="\t")

Used for some exmaples below

In [4]:
def align(seq1,seq2):
    aligner = Align.PairwiseAligner()
    aligner.mode = "local"
    aligner.open_gap_score = -4
    aligner.extend_gap_score = -2
    aligner.match = 2
    aligner.mismatch = -3
    score = aligner.score(seq1, seq2)
    return score 

In [18]:
def jaccard_similarity(x,y):
    inter = len(set.intersection(set(x), set(y)))
    return inter / float(len(set(x)) + len(set(y)) - inter)

Compute the motifs for two example lists (should be the expansion lists)

In [6]:
at = FindMotifs(sample_list_at)
vv = FindMotifs(sample_list_vv)

Compute some values as testing.<br>
sim_j: matrix 5x5 with only jaccard distances<br>
sim_a: matrix 5x5 with only alignment<br>
sim: matrix 5x5 with jaccard*alignmet<br>

In [19]:
sim_j = np.zeros((5,5))
for i in range(len(at)):
    for j in range(len(vv)):
        sim_j[i,j] = jaccard_similarity(at[i],vv[j])

In [8]:
sim_a = np.zeros((5,5))
for i in range(5):
    for j in range(5):
        a = at_cdna.loc[(at_cdna["a_thaliana_gene"] == sample_list_at[i]),"a_sequence"].iloc[0]
        v = vv_cdna.loc[(vv_cdna["vitis_gene"] == sample_list_vv[j]),"v_sequence"].iloc[0]
        sim_a[i,j] = align(a,v)

In [20]:
sim = np.zeros((5,5))
for i in range(len(at)):
    for j in range(len(vv)):
        a = at_cdna.loc[(at_cdna["a_thaliana_gene"] == sample_list_at[i]),"a_sequence"].iloc[0]
        v = vv_cdna.loc[(vv_cdna["vitis_gene"] == sample_list_vv[j]),"v_sequence"].iloc[0]
        sim[i,j] = jaccard_similarity(at[i],vv[j]) * align(a,v)

In [21]:
print("Jaccard")
print(sim_j)
print("Align")
print(sim_a)
print("Both")
print(sim)

Jaccard
[[0.31391586 0.2751938  0.18373494 0.28023599 0.15555556]
 [0.26719577 0.30097087 0.20725389 0.2835443  0.20718232]
 [0.25336927 0.21630094 0.24517906 0.29396325 0.25892857]
 [0.29807692 0.36666667 0.20245399 0.28106509 0.23890785]
 [0.29512894 0.26689189 0.19945355 0.40762463 0.36666667]]
Align
[[ 191.   31.   28.   34.   23.]
 [  26.  244.   28.   29.   28.]
 [  25.   35.  255.   28.   26.]
 [  31.   28.   27.  390.   28.]
 [  30.   27.   29.   29. 1163.]]
Both
[[ 59.9579288    8.53100775   5.14457831   9.5280236    3.57777778]
 [  6.94708995  73.4368932    5.80310881   8.22278481   5.80110497]
 [  6.33423181   7.57053292  62.52066116   8.23097113   6.73214286]
 [  9.24038462  10.26666667   5.46625767 109.61538462   6.6894198 ]
 [  8.85386819   7.20608108   5.78415301  11.82111437 426.43333333]]


Compute the ratio between the max and the other of a line of the matrix to see if things get better from sim_j/a to sim

In [29]:
print("Jaccard")
print(sim_j[4,:] / max(sim_j[4,:]))
print("Align")
print(sim_a[4,:] / max(sim_a[4,:]))
print("Both")
print(sim[4,:] / max(sim[4,:]))

Jaccard
[0.72402136 0.65474917 0.48930692 1.         0.89952038]
Align
[0.02579536 0.02321582 0.02493551 0.02493551 1.        ]
Both
[0.02076261 0.01689849 0.01356403 0.0277209  1.        ]


Test for task

In [27]:
task_at = FindMotifs([sample_task_at])
task_vv = FindMotifs(sample_task_vv)

In [28]:
print(jaccard_similarity(task_at[0],task_vv[0]))
print(jaccard_similarity(task_at[0],task_vv[1]))
print(jaccard_similarity(task_at[0],task_vv[2]))

0.24838012958963282
0.4149184149184149
0.3048245614035088
